# Generate upsetplots per tissue for dataset comparison

In [ ]:
library(ComplexHeatmap)
library(dplyr)

## Functions

In [ ]:
subset_data <- function(tissue, direction, method){
    dt = data.table::fread("../../_m/clinical_overlap_ancestryDEGs.txt.gz") %>%
        filter(Tissue == tissue, Direction == direction, Method == method) %>%
        select(Dataset, Genes) #%>% tibble::column_to_rownames("Dataset")
    return(dt)
}

In [ ]:
get_annotation <- function(tissue, direction, method){
    ta = c(200, 60, 200, 250, 100, 40, 75, 125, 150, 40, 125, 150,
           100, 20, 75, 100, 50, 20, 40, 50, 50, 20, 40, 40)
    ra = c(500, 150, 500, 600, 250, 75, 250, 300, 300, 100, 300, 300, 
           150, 30, 150, 150, 75, 20, 75, 75, 75, 20, 75, 75)
    annot = data.table::fread("../../_m/clinical_overlap_ancestryDEGs.txt.gz") %>% 
        select(-c(Genes, Dataset)) %>% distinct %>% mutate(TA=ta, RA=ra) %>%
        filter(Tissue == tissue, Direction == direction, Method == method) %>%
        select(TA, RA)
    return(annot)
}

get_data <- function(tissue, direction, method){
    datasets = c('BS_Caudate_SZ','BS_DLPFC_SZ','BS_Hippocampus_SZ','CMC_DLPFC_SZ', 'PSY_SZ',
             'BS_Amyg_BD', 'BS_sACC_BD', 'PSY_BD', 'Parkinsons', 'PSY_ASD',
             'MAYO_AD', 'MSBB_MD10_AD','MSBB_MD22_AD','MSBB_MD36_AD','MSBB_MD44_AD','ROSMAP_AD')
    lt = list()
    for(dataset in datasets){
        #print(dataset)
        dt = subset_data(tissue, direction, method) %>% filter(Dataset == dataset)
        lt[[dataset]] <- dt$Genes
    }
    m = make_comb_mat(lt)
    return(m)
}

plot_upsetplot <- function(tissue, direction, method){
    annot <- get_annotation(tissue, direction, method)
    datasets = c('BS_Caudate_SZ','BS_DLPFC_SZ','BS_Hippocampus_SZ','CMC_DLPFC_SZ', 'PSY_SZ',
             'BS_Amyg_BD', 'BS_sACC_BD', 'PSY_BD', 'Parkinsons', 'PSY_ASD',
             'MAYO_AD', 'MSBB_MD10_AD','MSBB_MD22_AD','MSBB_MD36_AD','MSBB_MD44_AD','ROSMAP_AD')
    m <- get_data(tissue, direction, method)
    cbb_palette <- ggpubr::get_palette(palette="npg", 16)
    label = gsub(" ", "_", paste(tolower(tissue), tolower(direction), tolower(method), sep="_"))
    fn = paste0(label, ".pdf")
    ## Annotate
    right_annot = upset_right_annotation(m, ylim = c(0, annot$RA), gp = gpar(fill = "black"),
                                         annotation_name_side = "top", axis_param = list(side = "top"))
    top_annot = upset_top_annotation(m, height=unit(8, "cm"), ylim = c(0, annot$TA),
                                     gp=gpar(fill=cbb_palette[comb_degree(m)]), annotation_name_rot = 90)
    ## Save plot as PDF
    pdf(fn, width=18, height=6)
    ht = draw(UpSet(m, pt_size=unit(3, "mm"), lwd=3, comb_col=cbb_palette[comb_degree(m)], 
                    set_order = datasets, comb_order = order(-comb_size(m)), 
                    row_names_gp = gpar(fontsize = 12, fontface='bold'),
                    right_annotation = right_annot, top_annotation = top_annot))
    od = column_order(ht); cs = comb_size(m)
    decorate_annotation("intersection_size", {
        grid.text(cs[od], x = seq_along(cs), y = unit(cs[od], "native") + unit(6, "pt"), 
        default.units = "native", just = "bottom", gp = gpar(fontsize = 9))
    })
    dev.off()
}

## Main

In [ ]:
for(tissue in c("Caudate", "Dentate Gyrus", "DLPFC", "Hippocampus")){
    for(direction in c("All", "AA Bias", "EA Bias")){
        for(method in c("DEG", "TWAS")){
            plot_upsetplot(tissue, direction, method)
        }
    }
}

## Reproducibility Information

In [ ]:
Sys.time()
proc.time()
options(width = 120)
sessioninfo::session_info()